###  <a href='https://ko-fi.com/P5P1QO6VB' target='_blank'><img height='36' style='border:0px;height:36px;' src='https://storage.ko-fi.com/cdn/kofi4.png?v=3' border='0' alt='Buy Me a Coffee at ko-fi.com' /></a>

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="https://i.ibb.co/w6ngBsW/20231031152934.jpg" alt="20231031152934" width="300">
    <span style="margin: 0 20px;">|</span>
    <img src="https://i.ibb.co/SrxYKmz/20231031152928.jpg" alt="20231031152928" width="400">
</div>

## VoltaML_colab

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/voltaml/voltaml.ipynb)
[![Visitors](https://api.visitorbadge.io/api/visitors?path=https%3A%2F%2Fcolab.research.google.com%2Fgithub%2FVan-wise%2Fsd-colab%2Fblob%2Fmain%2Fvoltaml%2Fvoltaml.ipynb&labelColor=%232ccce4&countColor=%23555555&style=flat&labelStyle=upper)](https://visitorbadge.io/status?path=https%3A%2F%2Fcolab.research.google.com%2Fgithub%2FVan-wise%2Fsd-colab%2Fblob%2Fmain%2Fvoltaml%2Fvoltaml.ipynb)

---


<font color="red">**更新地址**：https://github.com/Van-wise/sd-colab

**如果遇到报错,请联系up：**
[bilibili](https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244) or [issues](https://github.com/Van-wise/sd-colab/issues)



In [ ]:
# @title (🔆)安装环境
import os
import time
from IPython.utils import capture
from IPython.display import clear_output, display, HTML

# 忽略警告
import warnings
os.environ['PYTHONWARNINGS'] = 'ignore'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
!git config --global advice.detachedHead false
!sed -i 's@text = _formatwarnmsg(msg)@text =\"\"@g' {warnings.__file__}
warnings.filterwarnings("ignore")

def check_gpu():
    import tensorflow as tf
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
      print("\033[96m GPU is available.\033[0m", flush=True)
    else:
      print("\033[91m 没有使用GPU,请在代码执行程序-更改运行时类型-设置为GPU!\033[0m", flush=True)
      display(HTML("<img src='https://i.ibb.co/xfb7pB7/check-gpu.png' width='560px'/>"))
      from google.colab import runtime
      runtime.unassign()

%cd /content
#@markdown ###<font color="#11659a"> 选择版本：</font>
_branch = "main" # @param ["v0.5.0", "v0.4.2", "main"]
#@markdown #####<font color="#2c9678">☑检查是否使用了GPU；☐不检查。
_check_gpu = True  #@param {type:"boolean"}
# @markdown #####填写：[HUGGINGFACE_TOKEN](https://huggingface.co/settings/tokens)
hf_token = "1" # @param {type:"string"}
os.environ['HUGGINGFACE_TOKEN'] = hf_token

if _check_gpu:
  check_gpu()
start_time = time.time()

!git clone -q -b {_branch} https://github.com/VoltaML/voltaML-fast-stable-diffusion
print('项目克隆成功！')
main_dir = "/content/voltaML-fast-stable-diffusion"
print("正在安装环境,大约需要三分钟...")

%cd {main_dir}
with capture.capture_output() as cap:
  !apt-get -qq -y update
  !apt -y install -qq aria2
  !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M https://github.com/Van-wise/sd-colab/raw/main/voltaml/wise.py -d /content -o wise.py
  %run /content/wise.py
  fix_colab(main_dir)
  !pip uninstall -y lida llmx
  !pip install -q kaleido cohere openai tiktoken jedi
  !python main.py --install-only
end_time = time.time()
print("环境安装完成，耗时：{:.3f}分钟.".format((end_time - start_time) / 60))

###  🌐下载模型

In [ ]:
# @title (🔆)MOD！
# @markdown #### 💠用于下载主模型、Lora、Vae、Embedding.
import pandas as pd
import requests
from io import BytesIO
from google.colab import files, drive
from concurrent import futures

# @markdown #### 启用MOD:
enable_mod = True #@param {type:"boolean"}
# @markdown #### 模型下载位置:
modown_dir = "colab" # @param ["colab", "drive"]
if modown_dir == "drive":
  modelr_dir = "/content/drive/MyDrive/sd_models"
else:
  modelr_dir = f"{main_dir}/data/"
# @markdown ---
# @markdown ##### [MOD链接](https://docs.google.com/spreadsheets/d/1aRQNNgCpnDZt_mLOuBOAjeoz-osm_7MS/edit#gid=149213849):
mod_link = "https://docs.google.com/spreadsheets/d/e/2PACX-1vS0Jie7yPe6EDJDYepjXxH7muvEib-linridXHWNDC9Y09Wjp6zeY4JmBTQdwke0w/pub?output=xlsx"# @param {type:"string"}
# @markdown ##### MOD路径:
mod_path = "/content/drive/MyDrive/wise.xlsx"# @param {type:"string"}
# @markdown ##### 代码逻辑:优先使用MOD链接,链接为空使用MOD路径,路径为空使用上传MOD。

if enable_mod:
  !apt-get -qq install -y aria2
  start_mod(mod_link,mod_path,modelr_dir)

In [7]:
# @title 内网穿透
import os
import shlex
import subprocess
from pathlib import Path
from typing import Union
# @markdown #### [Ngrok](https://dashboard.ngrok.com/get-started/your-authtoken):
use_ngrok = False # @param {type:"boolean"}
ngrok_token = "" # @param {type:"string"}
# @markdown #### cloudflare:
use_cloudflare = True # @param {type:"boolean"}
# @markdown #### remote:
use_remote = True # @param {type:"boolean"}

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

if use_remote:
  id_rsa_file = "/content/id_rsa"
  id_rsa_pub_file = "/content/id_rsa/id_rsa.pub"
  if os.path.exists(id_rsa_file):
      os.remove(id_rsa_file)
  if os.path.exists(id_rsa_pub_file):
      os.remove(id_rsa_pub_file)
  ssh_name = "id_rsa"
  ssh_path = Path(os.path.dirname(os.getcwd())) / ssh_name
  gen_key(ssh_path)

if use_ngrok:
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_token)

def start_tunnle():
  #time.sleep(1)
  open("/content/output.log", "w").close()
  # 循环检测文件内容，直到存在"http://127.0.0.1:9090"。
  while not "numexpr.utils" in open("/content/output.log", "r").read():
    time.sleep(1)

  clear_output()
  from IPython.display import display, HTML
  audio_url = "https://github.com/Van-wise/sd-colab/raw/main/qidong.mp3"
  display(HTML(f'<audio src="{audio_url}" controls autoplay style="display:none"></audio>'))

  if use_ngrok:
      try:
          from pyngrok import ngrok
          ngrok_tunnel = ngrok.connect(5003, "http")
          print("ngrok_tunnel:", ngrok_tunnel)
      except Exception as e:
          print("ngrok 连接失败：", e)

  if use_cloudflare:
      try:
          from pycloudflared import try_cloudflare
          cloudflare_url = try_cloudflare(5003, verbose=False)
          print("cloudflare_tunnel:", cloudflare_url)
      except Exception as e:
          print("cloudflare 连接失败：", e)

  if use_remote:
      !ssh -R 80:127.0.0.1:5003 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe 2>&1 | tee -a /content/tunnel.log

### (⚒)附加功能：

In [ ]:
# @markdown #### 🌐使用云盘模型:
use_drivemodels = False # @param {type:"boolean"}
gmodelr_dir = "/content/drive/MyDrive/sd_models" # @param ["/content/drive/MyDrive/sd_models"] {allow-input: true}
# @markdown #### 📁打包输出图片:
use_zipoutputs = True # @param {type:"boolean"}
#@markdown #####保存间隔（秒）：
save_interval = 120 #@param {type:"integer"}
#@markdown #####压缩保存目录：
zip_path = "/content/outputs" # @param ["/content/outputs", "/content/drive/MyDrive/outputs"] {allow-input: true}

if zip_path == "/content/drive/MyDrive/outputs":
  if not os.path.exists('/content/drive/'):
    from google.colab import drive
    drive.mount('/content/drive')
    print('Google Drive 挂载成功！')
    os.makedirs('/content/drive/MyDrive/outputs', exist_ok=True)


import os
def content_models():
  if not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    print('Google Drive 挂载成功！')

  for modelver in ["models", "lora", "lycoris", "vae", "onnx"]:
      sub_dir_path = f"{gmodelr_dir}/{modelver}"
      os.makedirs(sub_dir_path, exist_ok=True)

      if len(os.listdir(sub_dir_path)) == 0:
          continue
      os.system(f"ln -sf {sub_dir_path}/* {main_dir}/data/{modelver}")

import shutil
import time
from datetime import datetime

def zip_outputs(outputs_path, zip_path, save_interval):
  save_count = 0
  last_backup_path = None

  while True:
    time.sleep(save_interval)

    save_count += 1
    backup_name = f"{datetime.now().strftime('%m_%d')}_outputs_{datetime.now().strftime('%H:%M')}_{save_count}"
    backup_path = f"{zip_path}/{backup_name}"

    if last_backup_path:
      os.remove(last_backup_path)

    try:
      shutil.make_archive(backup_path, 'zip', root_dir=outputs_path)
      print(f'输出图片备份成功: {backup_path}.zip')
      last_backup_path = f"{backup_path}.zip"
    except Exception as e:
      print(f"创建备份时出现错误：{e}")

import multiprocessing
def start_background_task(target_func, *args):
  p = multiprocessing.Process(target=target_func, args=args)
  p.start()
  return p

def stop_background_task(process):
  process.terminate()

if use_drivemodels:
  content_models()

In [ ]:
# @title 启动
import threading
thread = threading.Thread(target=start_tunnle, daemon=True)
thread.start()

if 'use_zipoutputs' in globals():
    if use_zipoutputs:
        outputs_path = f"{main_dir}/data/outputs"
        back_zip = start_background_task(zip_outputs, outputs_path, zip_path, save_interval)
else:
    pass

!python main.py 2>&1 | tee /content/output.log

### 关闭进程

In [ ]:
!ps -ef #显示进程
!pkill -f server.py
!pkill -f pyngrok
!pkill -f cloudflared
#杀进程